A ideia básica é treinar um classificador utilizando cross-validation 5-folds. Vou realizar o corte em folds no .csv e dentro do loop utilizar dataloaders.

In [1]:
import torch
import pandas as pd
import os
import imageio as im
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,utils
import torchvision
import numpy as np
import time
import copy
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.model_selection import KFold

In [2]:
import sys

sys.path.append('/store/vrusso/skin-tone-research/tools')
for p in sys.path:
    print( p )
    
from fitzpatrick17k import FitzpatrickDataset

/store/vrusso/skin-tone-research/simple_cnn_classifier
/opt/conda/lib/python38.zip
/opt/conda/lib/python3.8
/opt/conda/lib/python3.8/lib-dynload

/opt/conda/lib/python3.8/site-packages
/store/vrusso/skin-tone-research/tools


In [3]:
df = pd.read_csv('../../datasets/fitzpatrick17k/fitzpatrick17k.csv')

df

,md5hash,fitzpatrick,label,nine_partition_label,three_partition_label,url,url_alphanum
0,dae2d6176ea28b4d991c573c8320b0cf,4,lichen planus,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=3739,httpwwwatlasdermatologicocombrimgimageId3739.jpg
1,dcba3efe0fe28be2b0032df9470c320d,5,behcets disease,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=8633,httpwwwatlasdermatologicocombrimgimageId8633.jpg
2,9a9ded2de5e2c20a90cdfc2971e2e155,2,pityriasis lichenoides chronica,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppityr...
3,d6166c9947e7e99915ff4d6907ddec44,2,mycosis fungoides,malignant cutaneous lymphoma,malignant,http://atlasdermatologico.com.br/img?imageId=4594,httpwwwatlasdermatologicocombrimgimageId4594.jpg
4,8ea58508e89d61a98320acb03139b8bb,3,urticaria pigmentosa,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicuurtic...
...,...,...,...,...,...,...,...
16520,54aa2f1bf36b27de991db5cd358bde06,2,squamous cell carcinoma,malignant epidermal,malignant,http://atlasdermatologico.com.br/img?imageId=6834,httpwwwatlasdermatologicocombrimgimageId6834.jpg
16521,b3ce9d9d122db072fdf3245521263c06,2,erythema multiforme,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpiceeryth...
16522,6fcaf3c1ecef5a938bf70ae2b6ed24f6,2,basal cell carcinoma,malignant epidermal,malignant,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicbbasal...
16523,b0b5f5950d2fda0eed8304ea18eef430,4,actinic keratosis,malignant epidermal,malignant,http://atlasdermatologico.com.br/img?imageId=3067,httpwwwatlasdermatologicocombrimgimageId3067.jpg


In [4]:
print("Número de instâncias com -1 no label:"+str(len(df[df['fitzpatrick']== -1])))
print("Dropando estas instâncias...")
df = df.drop(df[df['fitzpatrick']== -1].index,axis=0)
df.head()

Número de instâncias com -1 no label:559
Dropando estas instâncias...


,md5hash,fitzpatrick,label,nine_partition_label,three_partition_label,url,url_alphanum
0,dae2d6176ea28b4d991c573c8320b0cf,4,lichen planus,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=3739,httpwwwatlasdermatologicocombrimgimageId3739.jpg
1,dcba3efe0fe28be2b0032df9470c320d,5,behcets disease,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=8633,httpwwwatlasdermatologicocombrimgimageId8633.jpg
2,9a9ded2de5e2c20a90cdfc2971e2e155,2,pityriasis lichenoides chronica,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppityr...
3,d6166c9947e7e99915ff4d6907ddec44,2,mycosis fungoides,malignant cutaneous lymphoma,malignant,http://atlasdermatologico.com.br/img?imageId=4594,httpwwwatlasdermatologicocombrimgimageId4594.jpg
4,8ea58508e89d61a98320acb03139b8bb,3,urticaria pigmentosa,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicuurtic...


In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=19 )
kf

KFold(n_splits=5, random_state=19, shuffle=True)

In [9]:
for train_index, test_index in kf.split(df):
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]
    
    data_train = FitzpatrickDataset( df_train,'../../datasets/fitzpatrick17k/resized_images', target = 'fitzpatrick',
                          transform = transforms.Compose([transforms.ToTensor(),
                                                          transforms.RandomResizedCrop(224),
                                                          transforms.RandomHorizontalFlip(),
                                                          transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]))
    
    train_dataloader = DataLoader(data_train,batch_size=64,num_workers=8,shuffle=True)
    
    data_val = FitzpatrickDataset( df_test,'../../datasets/fitzpatrick17k/resized_images', target = 'fitzpatrick',
                          transform = transforms.Compose([transforms.ToTensor(),
                                                          transforms.CenterCrop(224),
                                                          transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]))
    
    val_dataloader = DataLoader(data_val,batch_size=64,num_workers=8,shuffle=False)
    
    for batch in val_dataloader:
        pass
    
    print('alo')

alo
alo
alo
alo
alo
